In [2]:
import torch
import inspect

torch.__version__

'1.11.0+cu113'

## Practice

## Mnist model prediction

In [3]:
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# import matplotlib.pyplot as plt

test_data_path = './mnist_test.csv'

df_test = pd.read_csv(test_data_path)

batch_size = 250

device = 'cuda' if torch.cuda.is_available() else 'cpu'

_x = torch.Tensor(df_test.iloc[:,1:].values).reshape(len(df_test),28,28) / 256
_x = _x.unsqueeze(1)
_y = torch.Tensor(df_test.iloc[:,0].values).type(torch.long)
print(_y)

print(_x.shape)
print(_y.shape)

class DigitDataset(Dataset):
    def __init__(self, x, y=None):
        super(DigitDataset).__init__()
        self.x = x
        self.y = y
    
    def __getitem__(self, idx):
        if self.y == None:
            return self.x[idx]
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784,256),
            nn.ReLU(),
            nn.Linear(256,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forword(self,x):
        x = self.model(x)
        x = torch.softmax(x, dim=-1)
        return x


# mnist_net = Net()
model = torch.load('./mnist_net.pth')
l = len(df_test)
valid_x = _x
valid_y = _y
testset = DigitDataset(valid_x, valid_y)
test_dl = DataLoader(testset, batch_size = batch_size)
acc=0
model.eval()

for tx, ty in test_dl:
    # tx = tx.to(device)
    # ty = ty.to(device)

    print(ty)

    ty_hat = model(tx)
    test_pred = ty_hat.max(dim=1)[1]
    print(test_pred)

pred_y_raw = model.forword(_x)
pred_y = pred_y_raw.argmax(axis=1)

print(pred_y_raw[0,:])
print(pred_y)
print(_y)

err_function = nn.MSELoss()
acc = err_function(pred_y, _y)
print(acc)


tensor([2, 1, 0,  ..., 4, 5, 6])
torch.Size([9999, 1, 28, 28])
torch.Size([9999])
tensor([2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4,
        0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4,
        4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2,
        9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1,
        7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5,
        4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7,
        1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1,
        4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0,
        9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3,
        6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5,
        9, 8, 7, 2, 3, 0, 4, 4, 2, 4])


NotImplementedError: 